In [ ]:
# Goal: implement _very_ basic functionalities of pytorch from scratch (but I'm going to cheat and use numpy)

# large inspiration from:
# - https://github.com/karpathy/micrograd
# - https://github.com/geohot/tinygrad

In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

In [83]:
# pytorch

# Create tensors.
x = torch.tensor(15., requires_grad=True)
print(x)
w = torch.tensor(10., requires_grad=True)
b = torch.tensor(900., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3
print(y)

# Compute gradients.
# Computes the sum of gradients of given tensors with respect to graph leaves.
y.backward()

# Print out the gradients.
# Computes and returns the sum of gradients of outputs with respect to the inputs.
print(x.grad)    # x.grad = 2 = dy/dx = w
print(w.grad)    # w.grad = 1 = dy/dw = x
print(b.grad)    # b.grad = 1 = dy/db = 1

tensor(15., requires_grad=True)
tensor(1050., grad_fn=<AddBackward0>)
tensor(10.)
tensor(15.)
tensor(1.)


In [91]:
# not pytorch

class Tensor:
    def __init__(self, data, children=()):
        self.data = np.array(data, dtype=np.float32)
        self._children = children
        self.grad = 0
        self._backword = lambda: None
        
    def __mul__(self, other):
        out = Tensor(np.multiply(self.data, other.data), (self, other))
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
            
        out._backword = _backward
        
        return out
    
    def __add__(self, other):
        out = Tensor(np.add(self.data, other.data), (self, other))
        
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        
        return out
    
    def backward(self):
        graph = []
        
        def _build_graph(node):
            for child in node._children:
                _build_graph(child)
            graph.append(node)
        
        _build_graph(self)
        
        self.grad = 1
        for node in reversed(graph):
            node._backword()

        
x = Tensor([15])
print(f'x: {x.data}')
w = Tensor([10])
b = Tensor([900])

y = w * x + b
print(f'y: {y.data}')

y.backward()

print(x.grad)
print(w.grad)
print(b.grad)

x: [15.]
y: [150.]
[10.]
[15.]
